In [3]:
import cv2
import numpy as np
roi_hist_save = np.array([[  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [255.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.]], dtype=np.float32)

In [58]:
def run_ex_cam_gen(num, x, y, size, roi_hist):
    cap = cv2.VideoCapture('../vid/droning_on2{}.mp4'.format(num))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 3500)
    orb = cv2.ORB_create(nfeatures=300)
    #orb = cv2.ORB_create()
    #cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
    ret, frame = cap.read()
    scale = 2
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    whole = size // scale
    y, x = y // scale, x // scale
    crop = frame[y:y + whole, x:x + whole]
    r, h, c, w = y, whole, x, whole
    #r,h,c,w = 250,90,400,125 
    track_window = (c,r,w,h)
    
    kp_prev, des_prev = orb.detectAndCompute(crop,None)
    height,width, depth = crop.shape
    black = np.zeros((height,width, depth), np.uint8)
    for sing_kp in kp_prev:
        x, y = sing_kp.pt
        cv2.circle(black,(np.int(x),np.int(y)), 10, (0,255,0), -1)
    cv2.imshow('crop', crop)
    cv2.imshow('black', black)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
    
    i = 0
    while True:
        ret, frame = cap.read()
        if ret == False or cv2.waitKey(1) & 0xff == 27:
            break
        frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
        copy = frame.copy()
        kp, des = orb.detectAndCompute(frame,None)
        frame = cv2.drawKeypoints(frame,kp,None,color=(0,255,0), flags=0)
        height,width, depth = frame.shape
        black = np.zeros((height,width, depth), np.uint8)
        for sing_kp in kp:
            x, y = sing_kp.pt
            cv2.circle(black,(np.int(x),np.int(y)), 10, (0,255,0), -1)
        #cv2.imshow('frame', frame)
        #cv2.imshow('black', black)
        hsv = cv2.cvtColor(black, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
        tot_x, tot_y = 0, 0
        min_x, min_y = 0, 0
        max_x, max_y = 100, 100
        num = len(pts)
        if num != 0:
            for cur_x, cur_y in pts:
                if cur_x < min_x:
                    min_x = cur_x
                if cur_x > max_x:
                    max_x = cur_x
                if cur_y < min_y:
                    min_y = cur_y
                if cur_y > max_y:
                    max_y = cur_y
                tot_x += cur_x
                tot_y += cur_y
            px = tot_x // num
            py = tot_y // num
            dif_x = max_x - min_x
            dif_y = max_y - min_y
        
        out = copy[py - dif_y // 8:py + dif_y // 8, px - dif_x // 12:px + dif_x // 12]
        cv2.imwrite('droning_on/droning_on{}.jpg'.format(i), out)
        #print(i)
        cv2.imshow('out', out)
        cv2.imshow('img2',img2)
        i += 1
    cv2.destroyAllWindows()
    cap.release()
#1
run_ex_cam_gen(num='', x=800, y=300, size=150, roi_hist=roi_hist_save)

In [35]:
print('s')

s
